In [78]:
import numpy as np
import nltk
import os
import difflib
from gensim.models import Word2Vec
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.util import ngrams
import gensim.downloader as api
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
nltk.download('punkt')
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer() #lemmatizer algorithm
lancStemmer = LancasterStemmer()  # stemming algorithm Lancaster

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sergiogonzalez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sergiogonzalez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sergiogonzalez/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### PREPROCESAMIENTO
(gala)

In [79]:
def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palabras = [palabra.lower() for palabra in re.findall(r'\w+', text.lower())]
    text_lista = []
    for palabra in palabras:
        if palabra not in stopwords:
            text_lista.append(palabra)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

def get_lemmatizer(text):
    palabras = remove_stopwords(text)
    palabras = palabras.split()
    text_lista = []
    for palabra in palabras:
        nueva = lemmatizer.lemmatize(palabra)
        text_lista.append(nueva)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

def get_stemmer(text):
    palabras = remove_stopwords(text)
    palabras = palabras.split()
    text_lista = []
    for palabra in palabras:
        nueva = lancStemmer.stem(palabra)
        text_lista.append(nueva)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

In [80]:
def get_grams(text, ngram, method):
    result = []

    if method == 'lemmatize':
        text = get_lemmatizer(text)
        if ngram == 0:  # Si ngram es 0, se retorna el texto completo sin ngramas
            text = nltk.sent_tokenize(text)
            text = ' '.join(text)
            return text

        else:
            text = text.split()
            grams = ngrams(text, ngram)
            for ng in grams:
                result.append(' '.join(ng))
    elif method == 'stemmer':
        text = get_stemmer(text)
        if ngram == 0:  # Si ngram es 0, se retorna el texto completo sin ngramas
            text = nltk.sent_tokenize(text)
            text = ' '.join(text)
            return text

        else:
            text = text.split()
            grams = ngrams(text, ngram)
            for ng in grams:
                result.append(' '.join(ng))
    else:
        raise ValueError('Method not found')

    return result

### PROCESAMIENTO
Para documentos totales (sergio)

In [81]:
def preprocess_docs(folder_path, ngram, method):
    """
    Esta función recibe la ruta de una carpeta con documentos de texto 
    y retorna una lista de documentos preprocesados y taggeados para el modelo
    """
    tagged_documents = []
    lemmatizer = WordNetLemmatizer()
    for fileid in os.listdir(folder_path):
        if fileid.endswith(".txt"):
            filepath = os.path.join(folder_path, fileid)
            
            with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
                text = file.read()
                # Preprocesamiento de texto
                grams = get_grams(text, ngram, method)
                # Ensure words are split into a list of strings and then converted to tuple
                words = tuple(word.split() for word in grams)
                # Flatten the list of lists into a single list of strings
                words = [word for sublist in words for word in sublist if word.isalpha()]
                tagged_documents.append(TaggedDocument(words=words, tags=[fileid]))

    return tagged_documents

Obtiene los documentos tageados y sus palabras de los documentos preprocesadas en una listad de listas

In [82]:
folder_path = "../../final-testing"  # Ruta de la carpeta con los textos plagiados)
folder_path_og = "../../docs_originales"  # Ruta de la carpeta con los textos originales

tagged_originals = preprocess_docs(folder_path_og, 1, 'lemmatize')
tagged_plagiarized = preprocess_docs(folder_path, 1, 'lemmatize')  

## ENTRENAMIENTO DE MODELO 
aqui voy a poner solo los originales para entrenamiento

In [83]:
def train_doc2vec(tagged_documents):
    model = Doc2Vec(vector_size=80, window=5, min_count=1, epochs=200,
                    dm=0)  # dm=0 for distributed bag of words (DBOW) mode
    model.build_vocab(tagged_documents)
    model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [84]:
# Training the Doc2Vec model
model = train_doc2vec(tagged_originals + tagged_plagiarized)

### Función para calcular la similitud entre los conjuntos creando vectores con el modelo

In [85]:
def calculate_similarity_doc2vec(doc1, doc2, model):
    vec1 = model.infer_vector(doc1.words)
    vec2 = model.infer_vector(doc2.words)
    similarity = model.dv.similarity(doc1.tags[0], doc2.tags[0])
    return similarity

In [100]:
similarity_results = []

# Iterating over each plagiarized text
for plagio_doc in tagged_plagiarized:
    max_similarity = 0
    most_similar = ''
    most_similar_doc = ''

    # Comparing with each original document
    for original_doc in tagged_originals:
        similarity = calculate_similarity_doc2vec(plagio_doc, original_doc, model)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar = original_doc.tags[0]
            most_similar_doc = original_doc.words

    similarity_results.append([plagio_doc.tags[0], most_similar, max_similarity, most_similar_doc])

        

# Sorting results by similarity in descending order
similarity_results.sort(key=lambda x: x[2], reverse=True)

# Printing results
for result in similarity_results:
    plagio_title, original_title, similarity_score, original_doc = result
    print(f"Similarity between '{plagio_title}' and '{original_title}': {similarity_score * 100:.2f}%")

Similarity between 'FID-027.txt' and 'org-067.txt': 99.81%
Similarity between 'FID-023.txt' and 'org-024.txt': 99.78%
Similarity between 'FID-019.txt' and 'org-078.txt': 99.73%
Similarity between 'FID-018.txt' and 'org-014.txt': 98.69%
Similarity between 'FID-026.txt' and 'org-101.txt': 98.22%
Similarity between 'FID-010.txt' and 'org-091.txt': 97.65%
Similarity between 'FID-022.txt' and 'org-020.txt': 97.56%
Similarity between 'FID-016.txt' and 'org-057.txt': 96.00%
Similarity between 'FID-029.txt' and 'org-109.txt': 90.56%
Similarity between 'FID-005.txt' and 'org-059.txt': 83.39%
Similarity between 'FID-021.txt' and 'org-051.txt': 56.54%
Similarity between 'FID-015.txt' and 'org-014.txt': 51.78%
Similarity between 'FID-012.txt' and 'org-048.txt': 49.94%
Similarity between 'FID-009.txt' and 'org-070.txt': 49.56%
Similarity between 'FID-028.txt' and 'org-083.txt': 49.30%
Similarity between 'FID-020.txt' and 'org-055.txt': 48.51%
Similarity between 'FID-007.txt' and 'org-087.txt': 46.3

In [ ]:
similarity_results_all = []

# Iterando sobre cada texto plagiado
for plagio_doc in tagged_plagiarized:
    similarity_scores = []
    
    # Comparando con cada documento original
    for original_doc in tagged_originals:
        similarity = calculate_similarity_doc2vec(plagio_doc, original_doc, model)
        similarity_scores.append((original_doc.tags[0], similarity))

    # Ordenando los resultados por similitud en orden descendente
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Agregando los resultados a similarity_results
    similarity_results_all.append((plagio_doc.tags[0], similarity_scores))

# Imprimiendo los resultados
for plagio_title, similarity_scores in similarity_results_all:
    print(f"Nombre de documento plagiado: {plagio_title}")
    for original_title, similarity_score in similarity_scores:
        print(f"{original_title} --> {plagio_title} {similarity_score * 100:.2f}% de similitud")
    print()


## PROCESO POR ORACION 
(gala)

In [87]:
# Training the Doc2Vec model
model = train_doc2vec(tagged_originals + tagged_plagiarized)

In [88]:
def buscar_y_tokenizar(directorio, nombre_archivo):
    """
    Esta función recibe la ruta de un directorio y el nombre de un archivo
    y retorna una lista de oraciones tokenizadas del archivo
    ejemplo de salida: ['Primera oración.', 'Segunda oración.']
    """
    for filename in os.listdir(directorio):
        if filename == nombre_archivo:
            filepath = os.path.join(directorio, filename)
            with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
                text = file.read()
                sentences = nltk.sent_tokenize(text)
                return sentences
    return None

## TIPOS DE PLAGIO

In [89]:


def detector_cambio_voz(oracion_original, oracion_plagiada):
    """ 
    Detecta si hay cambio de voz entre dos oraciones al identificar si el sujeto de la oración cambió
    en la oración plagiada respecto a la oración original.
    """
    original_tags = nltk.pos_tag(word_tokenize(oracion_original))
    plagio_tags = nltk.pos_tag(word_tokenize(oracion_plagiada))
    
    # Identificar el sujeto en ambas oraciones
    for (word_orig, tag_orig), (word_plag, tag_plag) in zip(original_tags, plagio_tags):
        if tag_orig.startswith('VB') and tag_plag.startswith('VB'):
            if word_orig.lower() != word_plag.lower():  # Verificar si el verbo cambió
                return True  # Hay cambio de voz
    return False  # No hay cambio de voz



In [90]:
from difflib import SequenceMatcher

def detectar_reorganizacion(sentences_originales, sentences_plagiados, umbral_similitud=0.8):
    """
    Detecta reorganización de oraciones comparando la similitud entre las oraciones originales y plagiadas.
    """
    for sentence_orig, sentence_plagiada in zip(sentences_originales, sentences_plagiados):
        sentence_orig_text = ' '.join(sentence_orig.words)
        sentence_plagiada_text = ' '.join(sentence_plagiada.words)
        similitud = SequenceMatcher(None, sentence_orig_text, sentence_plagiada_text).ratio()
        if similitud >= umbral_similitud:
            return True  # Hay reorganización
    return False  # No hay reorganización



In [91]:

def detectar_cambio_tiempo2(oracion_original, oracion_plagiada):
    """
    Esta función detecta si hay un cambio de tiempo verbal en la oración plagiada en comparación con la original.
    Devuelve True si hay un cambio de tiempo verbal, de lo contrario, devuelve False.
    """
    # Etiquetar las palabras de ambas oraciones con sus partes del discurso (POS)
    tags_original = pos_tag(word_tokenize(oracion_original))
    tags_plagiada = pos_tag(word_tokenize(oracion_plagiada))
    print("Etiquetas parte palabra ORIGINAL: ",tags_original)
    print("Etiquetas parte palabra PLAGIADA: ",tags_plagiada)
    
    # Extraer los verbos de cada oración
    verbos_original = [palabra for palabra, etiqueta in tags_original if etiqueta.startswith('VB')]
    verbos_plagiada = [palabra for palabra, etiqueta in tags_plagiada if etiqueta.startswith('VB')]
    print("Verbos ORIGINAL: ",verbos_original)
    print("Verbos PLAGIADA: ",verbos_plagiada)
    
    # Verificar si hay cambios de tiempo verbal comparando los verbos de ambas oraciones
    if verbos_original and verbos_plagiada:
        tiempo_original = nltk.pos_tag(verbos_original)[0][1]
        tiempo_plagiada = nltk.pos_tag(verbos_plagiada)[0][1]
        return tiempo_original != tiempo_plagiada
    else:
        # Si no hay verbos en alguna de las oraciones, no se puede determinar un cambio de tiempo verbal
        return False

In [92]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def detectar_cambio_tiempo(oracion_original, oracion_plagiada):
    """
    Esta función detecta si hay un cambio de tiempo verbal en la oración plagiada en comparación con la original.
    Devuelve True si hay un cambio de tiempo verbal, de lo contrario, devuelve False.
    """
    # Inicializar lematizador y lista de stopwords
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Tokenizar las oraciones
    tokens_original = word_tokenize(oracion_original)
    tokens_plagiada = word_tokenize(oracion_plagiada)

    # Lematizar y filtrar palabras
    lemmatized_original = [lemmatizer.lemmatize(word) for word in tokens_original if word.isalnum() and word.lower() not in stop_words]
    lemmatized_plagiada = [lemmatizer.lemmatize(word) for word in tokens_plagiada if word.isalnum() and word.lower() not in stop_words]

    # Etiquetar las palabras de ambas oraciones con sus partes del discurso (POS)
    tags_original = pos_tag(lemmatized_original)
    tags_plagiada = pos_tag(lemmatized_plagiada)

    # Extraer los verbos de cada oración
    verbos_original = [palabra for palabra, etiqueta in tags_original if etiqueta.startswith('VB')]
    verbos_plagiada = [palabra for palabra, etiqueta in tags_plagiada if etiqueta.startswith('VB')]

    # Verificar si hay cambios de tiempo verbal comparando los verbos de ambas oraciones
    if verbos_original and verbos_plagiada:
        tiempo_original = nltk.pos_tag(verbos_original)[0][1]
        tiempo_plagiada = nltk.pos_tag(verbos_plagiada)[0][1]
        return tiempo_original != tiempo_plagiada
    else:
        # Si no hay verbos en alguna de las oraciones, no se puede determinar un cambio de tiempo verbal
        return False


In [94]:

def detectar_parafraseo(oracion_original, oracion_plagiada, umbral_similitud=0.8):
    """
    Esta función detecta si el tipo de plagio es parafraseo comparando la similitud entre las oraciones original y plagiada.
    Devuelve True si la similitud es mayor o igual al umbral especificado, de lo contrario, devuelve False.
    """
    # Calcular la similitud entre las oraciones original y plagiada
    similitud = SequenceMatcher(None, oracion_original, oracion_plagiada).ratio()
    
    # Comprobar si la similitud supera el umbral especificado
    return similitud >= umbral_similitud


In [95]:
def detectar_insertar_reemplazar(oracion_original, oracion_plagiada, umbral_palabras_comunes=0.8, umbral_longitud=0.5):
    """
    Esta función detecta si el tipo de plagio implica la inserción o reemplazo de frases del documento original en el documento plagiado.
    Devuelve True si la proporción de palabras comunes entre las oraciones supera el umbral especificado y la longitud de la oración plagiada es significativamente mayor que la original, de lo contrario, devuelve False.
    """
    # Tokenizar las oraciones en palabras
    palabras_originales = oracion_original.split()
    palabras_plagiadas = oracion_plagiada.split()

    # Calcular la proporción de palabras comunes
    palabras_comunes = set(palabras_originales) & set(palabras_plagiadas)
    prop_palabras_comunes = len(palabras_comunes) / len(set(palabras_originales))

    # Calcular la longitud relativa de la oración plagiada con respecto a la original
    long_orig = len(palabras_originales)
    long_plagiada = len(palabras_plagiadas)
    prop_longitud = long_plagiada / long_orig

    # Comprobar si la proporción de palabras comunes supera el umbral y la longitud de la oración plagiada es significativamente mayor que la original
    return prop_palabras_comunes >= umbral_palabras_comunes and prop_longitud >= umbral_longitud


# MAIN PROCESO POR ORACION

In [ ]:

from numpy import insert


def encontrar_coincidencias(sentences_originales, sentences_plagiados, model):
    coincidencias = []
    tipo_plagio = {'Cambio de voz': 0, 'Desorden de oraciones': 0, 'Cambio de tiempo verbal': 0, 'Parafraseo': 0, 'Inserción o reemplazo': 0}
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for sentence_orig in sentences_originales:
        tiene_coincidencia = False
        for sentence_plag in sentences_plagiados:
            similarity = calculate_similarity_doc2vec(sentence_orig, sentence_plag, model)
            if similarity > 0.6 and abs(len(sentence_orig) - len(sentence_plag)) < 15:
                coincidencias.append({
                    "cadena_orig": sentence_orig,
                    "cadena_plag": sentence_plag,
                    "similitud": similarity
                })
                if sentence_orig == sentence_plag:
                    TP += 1
                else:
                    FP += 1
                tiene_coincidencia = True


                cambio_voz = detector_cambio_voz(' '.join(sentence_orig.words), ' '.join(sentence_plag.words))
                #print("ORACION ORIGINAL:",' '.join(sentence_orig.words))
                #print("ORACION PLAGIADA:",' '.join(sentence_plag.words))
                reorganizacion = detectar_reorganizacion(sentences_originales, sentences_plagiados)
                cambio_tiempo = detectar_cambio_tiempo(' '.join(sentence_orig.words), ' '.join(sentence_plag.words))
                parafraseo = detectar_parafraseo(' '.join(sentence_orig.words), ' '.join(sentence_plag.words))
                insertar_reemplazar = detectar_insertar_reemplazar(' '.join(sentence_orig.words), ' '.join(sentence_plag.words))
                if cambio_voz:
                    tipo_plagio['Cambio de voz'] += 1
                if reorganizacion:
                    tipo_plagio['Desorden de oraciones'] += 1
                if cambio_tiempo:
                    tipo_plagio['Cambio de tiempo verbal'] += 1
                if parafraseo:
                    tipo_plagio['Parafraseo'] += 1
                if insertar_reemplazar:
                    tipo_plagio['Inserción o reemplazo'] += 1
                    
        #print(tipo_plagio)    
        if not tiene_coincidencia:
            FN += 1
        else:
            TN += 1
    # Contabilizar tipos de plagio y encontrar el mayor
    plagio_count = tipo_plagio
    #mayor_tipo_plagio = max(plagio_count, key=plagio_count.get)
    max_count = max(plagio_count.values())
    mayor_tipo_plagio = [tipo for tipo, count in plagio_count.items() if count == max_count]

    matriz_auc = {'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN}
    print(matriz_auc)
    print(f"Tipos de plagio contabilizados: {plagio_count}")
    print(f"Mayor tipo de plagio: {mayor_tipo_plagio}")
    return coincidencias, matriz_auc, mayor_tipo_plagio

In [96]:
def preprocess_sentences(sentences):
    """ 
    Función para preprocesar las oraciones.
    Después de tokenizar las oraciones:
    Se eliminan las stopwords y se lematizan las palabras.
    Se tagean las oraciones con un identificador único.
    """
    tagged_sentences = []
    lemmatizer = WordNetLemmatizer()
    for i, sentence in enumerate(sentences):
        tagged_sentence = TaggedDocument(words=[lemmatizer.lemmatize(word) for word in nltk.word_tokenize(sentence.lower()) if word.isalpha()], tags=[str(i)])
        tagged_sentences.append(tagged_sentence)
    return tagged_sentences

In [133]:
""" 
Ejecución del detector de plagio con el modelo Doc2Vec entrenado y las funciones de tipo de detección de plagio.
Se calculan las métricas de TPR, FPR y AUC.
"""
total_coincidencias = []
new_results = []
total_TP = 0
total_FP = 0
total_TN = 0
total_FN = 0

for titulo in similarity_results:
    resultados = []
    sentences_originales = buscar_y_tokenizar(folder_path_og, titulo[1])
    sentences_plagiados = buscar_y_tokenizar(folder_path, titulo[0])
    
    print(f"Titulo: {titulo[0]}")

    if sentences_originales and sentences_plagiados:
        tagged_sentences_originales = preprocess_sentences(sentences_originales)
        tagged_sentences_plagiados = preprocess_sentences(sentences_plagiados)
        model = train_doc2vec(tagged_sentences_originales + tagged_sentences_plagiados)
        similitud = titulo[2]
        print(f"Similitud entre '{titulo[0]}' y '{titulo[1]}': {similitud * 100:.2f}%")
        coincidencias, matriz_auc, tipo_plagio = encontrar_coincidencias(tagged_sentences_originales, tagged_sentences_plagiados, model)
        total_TP += matriz_auc['TP']
        total_FP += matriz_auc['FP']
        total_TN += matriz_auc['TN']
        total_FN += matriz_auc['FN']
        print(f"Tipo de plagio: {tipo_plagio}")
        total_coincidencias.extend(coincidencias)
        new_results.append([titulo[0], titulo[1], similitud, tipo_plagio])
        print()

        # print(f"\nCoincidencias para '{titulo[0]}' y '{titulo[1]}':")
        # for coincidencia in coincidencias:
            # print(f"Cadena original: {coincidencia['cadena_orig']} (Similitud: {coincidencia['similitud']})")
            # print(f"Cadena plagiada: {coincidencia['cadena_plag']}")
            # print()
    # else:
        # print(f"No se encontraron oraciones en '{titulo[0]}' o '{titulo[1]}'")
        # print()
    # print("----------------------------\n")

# Calculando TPR, FPR y AUC
TPR = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0
FPR = total_FP / (total_FP + total_TN) if (total_FP + total_TN) != 0 else 0
AUC = (1 + TPR - FPR) / 2

# Imprimiendo los valores calculados
print(f"TPR (Tasa de Verdaderos Positivos): {TPR:.2f}")
print(f"FPR (Tasa de Falsos Positivos): {FPR:.2f}")
print(f"AUC (Área bajo la curva ROC): {AUC:.2f}")

Titulo: FID-027.txt
Similitud entre 'FID-027.txt' y 'org-067.txt': 99.81%
{'TP': 8, 'FP': 2, 'TN': 10, 'FN': 0}
Tipos de plagio contabilizados: {'Cambio de voz': 2, 'Desorden de oraciones': 10, 'Cambio de tiempo verbal': 0, 'Parafraseo': 9, 'Inserción o reemplazo': 10}
Mayor tipo de plagio: ['Desorden de oraciones', 'Inserción o reemplazo']
Tipo de plagio: ['Desorden de oraciones', 'Inserción o reemplazo']

Titulo: FID-023.txt
Similitud entre 'FID-023.txt' y 'org-024.txt': 99.78%
{'TP': 6, 'FP': 14, 'TN': 12, 'FN': 0}
Tipos de plagio contabilizados: {'Cambio de voz': 8, 'Desorden de oraciones': 20, 'Cambio de tiempo verbal': 0, 'Parafraseo': 10, 'Inserción o reemplazo': 10}
Mayor tipo de plagio: ['Desorden de oraciones']
Tipo de plagio: ['Desorden de oraciones']

Titulo: FID-019.txt
Similitud entre 'FID-019.txt' y 'org-078.txt': 99.73%
{'TP': 0, 'FP': 13, 'TN': 5, 'FN': 0}
Tipos de plagio contabilizados: {'Cambio de voz': 0, 'Desorden de oraciones': 0, 'Cambio de tiempo verbal': 8, 'Pa

In [112]:
for resultado in new_results:
    copia = "Si" if resultado[2] > 0.8 else "No"
    resultado.insert(2, copia)

In [130]:
def imprimir_resultados(resultados):
    print("{:<50} | {:<10} | {:<50} | {:<7} | {:<20}".format("Documento sospechoso (plagiado)", "Copia", "Documento plagiado (original)", "% plagio", "Tipo de plagio"))
    print("-" * 130)
    for resultado in resultados:
        plagio_title = resultado[0]
        copia = resultado[2]
        original_title = resultado[1]
        similarity_score = resultado[3] * 100  # Convertir a porcentaje
        tipo_plagio = str(resultado[4])
        print("{:<50} | {:<10} | {:<50} | {:<7.2f} | {:<20}".format(plagio_title, copia, original_title, similarity_score, tipo_plagio))

In [131]:
# Usar la función para imprimir los resultados
imprimir_resultados(new_results)

Documento sospechoso (plagiado)                    | Copia      | Documento plagiado (original)                      | % plagio | Tipo de plagio      
----------------------------------------------------------------------------------------------------------------------------------
FID-027.txt                                        | Si         | org-067.txt                                        | 99.81   | ['Desorden de oraciones', 'Inserción o reemplazo']
FID-023.txt                                        | Si         | org-024.txt                                        | 99.78   | ['Desorden de oraciones']
FID-019.txt                                        | Si         | org-078.txt                                        | 99.73   | ['Cambio de tiempo verbal']
FID-018.txt                                        | Si         | org-014.txt                                        | 98.69   | ['Desorden de oraciones']
FID-026.txt                                        | Si         | org-1

In [108]:
# Crear una lista para almacenar los resultados
resultados = []

for titulo in similarity_results:
    plagio_title = titulo[0]
    original_title = titulo[1]
    similarity_score = titulo[2] * 100  # Convertir la similitud a porcentaje
    tipo_plagio = ', '.join(titulo[3]) if titulo[3] else "No detectado"  # Convertir la lista de tipos de plagio a una cadena, si existe
    copia = "Si" if similarity_score > 0.8 else "No"  # Determinar si es una copia basado en el umbral de similitud
    
    resultados.append((plagio_title, original_title, copia, similarity_score, tipo_plagio))

# Usar la función para imprimir los resultados
imprimir_resultados(new_results)


Documento sospechoso (plagiado)                    | Copia      | Documento plagiado (original)                      | % plagio | Tipo de plagio      
----------------------------------------------------------------------------------------------------------------------------------


IndexError: list index out of range

In [ ]:
!pip install pandas
!pip install openpyxl

In [ ]:
import pandas as pd

# Convertir los resultados en un DataFrame de pandas
df = pd.DataFrame(new_results, columns=["Documento sospechoso (plagiado)", "Copia", "Documento plagiado (original)", "% plagio", "Tipo de plagio"])

# Escribir el DataFrame en un archivo Excel
df.to_excel("resultados_plagio.xlsx", index=False)

In [ ]:
import os
from gensim.models.doc2vec import TaggedDocument
from nltk.stem import WordNetLemmatizer

# Definir la función de preprocesamiento de documentos
def preprocess_docs(folder_path, ngram, method):
    """
    Esta función recibe la ruta de una carpeta con documentos de texto 
    y retorna una lista de documentos preprocesados y taggeados para el modelo
    """
    tagged_documents = []
    lemmatizer = WordNetLemmatizer()
    for fileid in os.listdir(folder_path):
        if fileid.endswith(".txt"):
            filepath = os.path.join(folder_path, fileid)
            with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
                text = file.read()
                # Preprocesamiento de texto
                grams = get_grams(text, ngram, method)
                # Ensure words are split into a list of strings and then converted to tuple
                words = tuple(word.split() for word in grams)
                # Flatten the list of lists into a single list of strings
                words = [word for sublist in words for word in sublist if word.isalpha()]
                tagged_documents.append(TaggedDocument(words=words, tags=[fileid]))

    return tagged_documents

# Especificar las rutas de las carpetas de documentos originales y plagiados
folder_path_og = "../../docs_originales"  # Ruta de la carpeta con los textos originales
folder_path = "../../final-testing"  # Ruta de la carpeta con los textos plagiados

# Preprocesar los documentos originales y plagiados
tagged_originals = preprocess_docs(folder_path_og, 1, 'lemmatize')
tagged_plagiarized = preprocess_docs(folder_path, 1, 'lemmatize')  

# Entrenar el modelo Doc2Vec
model = train_doc2vec(tagged_originals + tagged_plagiarized)

# Definir una lista para almacenar los resultados de similitud
similarity_results_all = []

# Iterar sobre cada documento plagiado
for plagio_doc in tagged_plagiarized:
    similarity_scores = []

    # Comparar con cada documento original
    for original_doc in tagged_originals:
        similarity = calculate_similarity_doc2vec(plagio_doc, original_doc, model)
        similarity_scores.append((original_doc.tags[0], similarity))

    # Ordenar los resultados por similitud en orden descendente
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Agregar los resultados a similarity_results_all
    similarity_results_all.append((plagio_doc.tags[0], similarity_scores))

# Imprimir los resultados
for plagio_title, similarity_scores in similarity_results_all:
    print(f"Nombre de documento plagiado: {plagio_title}")
    for original_title, similarity_score in similarity_scores:
        print(f"{original_title} --> {plagio_title}: {similarity_score * 100:.2f}% de similitud")
    print()
